# 1. 추천 시스템 소개

이 노트북은 MovieLens 데이터셋을 사용한 영화 추천 시스템 구현에 관한 내용입니다. 협업 필터링과 행렬 분해 기법을 활용하여 사용자에게 개인화된 영화 추천을 제공하는 방법을 다룹니다.

## 1.1 필요한 라이브러리 불러오기

In [ ]:
# -*- coding: utf-8 -*-
# MovieLens ml-latest-small — 협업 필터링 과제용 통합 코드

import os
import sys
import io
import zipfile
import requests
import warnings
warnings.filterwarnings('ignore')

# 상위 디렉토리 경로를 시스템 경로에 추가하여 utils 모듈을 import할 수 있게 함
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

import dataclasses
from dataclasses import dataclass
from typing import Dict, List, Optional
from collections import defaultdict

import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.metrics.pairwise import cosine_similarity

# utils 모듈에서 필요한 클래스와 함수를 import
from utils.models import Dataset, RecommendResult, Metrics

## 1.2 핵심 데이터 구조 

추천 시스템에서 사용하는 핵심 데이터 구조를 살펴보겠습니다. 위에서 임포트한 `utils.models` 모듈에 다음 클래스들이 정의되어 있습니다:

1. **Dataset**: 추천 시스템 학습과 평가에 사용하는 데이터셋
   - `train`: 학습용 평점 데이터셋 (DataFrame)
   - `test`: 테스트용 평점 데이터셋 (DataFrame)
   - `test_user2items`: 순위 지표의 테스트 데이터셋 (Dict[int, List[int]])
   - `item_content`: 아이템 콘텐츠 정보 (DataFrame)

2. **RecommendResult**: 추천 시스템 예측 결과
   - `rating`: 테스트 데이터셋의 예측 평점, RMSE 평가용 (Series)
   - `user2items`: 사용자별 추천 아이템 목록, 순위 지표 평가용 (Dict[int, List[int]])

3. **Metrics**: 추천 시스템 평가 결과
   - `rmse`: Root Mean Squared Error
   - `precision_at_k`: Precision@K
   - `recall_at_k`: Recall@K
   - `params`: 추가 파라미터 (Dict)
   
이 데이터 구조들은 이후 모든 챕터에서 일관되게 사용됩니다.

In [ ]:
# 모듈이 정상적으로 로드되었는지 확인
print("Dataset 클래스:", Dataset)
print("RecommendResult 클래스:", RecommendResult)
print("Metrics 클래스:", Metrics)

# 예시 객체 생성
example_metrics = Metrics(rmse=0.95, precision_at_k=0.25, recall_at_k=0.35)
print("\n평가 메트릭스 예시:", example_metrics)